In [23]:
import glob
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.utils import shuffle
import pickle

In [24]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/NN Dataset/Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
from helpers import *

In [26]:
x_train = []
y_train = []

In [27]:
for filename in sorted(glob.glob('/content/drive/My Drive/NN Dataset/Females/*.jpg')):
    img = cv2.imread(filename)
    x_train.append(img)
    y_train.append(0)
for filename in sorted(glob.glob('/content/drive/My Drive/NN Dataset/Males/*.jpg')):
    img = cv2.imread(filename)
    x_train.append(img)
    y_train.append(1)

In [28]:
x_train, y_train = shuffle(x_train, y_train, random_state=25)
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [29]:
training_features_hog=[]
training_features_lbp=[]
training_features_glcm=[]
training_features_hinge=[]
for i in range(x_train.shape[0]):
    training_features_hog.append([])
    training_features_hog[i],_=findHOG(x_train[i])
    training_features_lbp.append([])
    training_features_lbp[i]=findLBP(x_train[i])
    training_features_glcm.append([])
    training_features_glcm[i]=findGLCM(x_train[i])
    training_features_hinge.append([])
    training_features_hinge[i]=findHINGE(x_train[i])

In [30]:
training_features=[]
for i in range(x_train.shape[0]):
  training_features.append([])
  training_features[i]=[*training_features_hog[i],*training_features_lbp[i],*training_features_glcm[i],*training_features_hinge[i]]

In [31]:
clf = svm.SVC(kernel='linear',probability=True)
clf.fit(training_features, y_train)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(training_features,y_train)
logreg = LogisticRegression(max_iter=300)
logreg.fit(training_features, y_train)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(training_features, y_train);

In [32]:
eclf = VotingClassifier(estimators=[('LR', logreg), ('RF', rfc), ('KNN', knn), ('SVC', clf)],voting='soft', weights=[1,1,1,2])
eclf.fit(training_features, y_train)

VotingClassifier(estimators=[('LR', LogisticRegression(max_iter=300)),
                             ('RF', RandomForestClassifier()),
                             ('KNN', KNeighborsClassifier(n_neighbors=3)),
                             ('SVC', SVC(kernel='linear', probability=True))],
                 voting='soft', weights=[1, 1, 1, 2])

In [38]:
pickle.dump(eclf, open("/content/drive/My Drive/NN Dataset/Project/model.sav", 'wb'))